In [1]:
from tensorflow import keras
from keras.models import Sequential
from keras.layers.merge import concatenate
from keras.models import Model
from keras.layers import Input, Dense, Conv2D, MaxPool2D , Flatten, Dropout, GlobalAveragePooling2D, AveragePooling2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import tensorflow as tf

2022-01-11 18:44:20.133022: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
Using TensorFlow backend.


In [2]:

tf.config.list_physical_devices('GPU')
#print("GPUs: ", len(tf.config.experimental.list_physical_devices('GPU')))

2022-01-11 18:44:21.617263: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-01-11 18:44:21.617977: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-01-11 18:44:21.666338: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-11 18:44:21.666810: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce GTX 1080 Ti computeCapability: 6.1
coreClock: 1.582GHz coreCount: 28 deviceMemorySize: 10.91GiB deviceMemoryBandwidth: 451.17GiB/s
2022-01-11 18:44:21.666835: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2022-01-11 18:44:21.668531: I tensorflow/stream_executor/platform/defau

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

rm/default/dso_loader.cc:49] Successfully opened dynamic library libcublasLt.so.11
2022-01-11 18:44:21.669245: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcufft.so.10
2022-01-11 18:44:21.669401: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcurand.so.10
2022-01-11 18:44:21.671057: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcusolver.so.10
2022-01-11 18:44:21.671453: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcusparse.so.11
2022-01-11 18:44:21.671557: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.8
2022-01-11 18:44:21.671637: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so r

In [3]:
print("GPUs: ", len(tf.config.experimental.list_physical_devices('GPU')))

GPUs:  1


In [4]:
train = pd.read_csv('train_metadata.csv')
test = pd.read_csv('test_metadata.csv')
#submission = pd.read_csv('sample_submission.csv')

train_dir = 'train/'
test_dir = 'test/'

In [5]:
label = []
train_data = []
for i in range(train.shape[0]):
    train_data.append(train_dir + train['image_name'].iloc[i] + '.jpg')
    label.append(train['diagnosis'].iloc[i])
    
test_data = []
for i in range(test.shape[0]):
    test_data.append(test_dir + test['image'].iloc[i] + '.jpg')

In [6]:
from sklearn.preprocessing import OneHotEncoder
one_hot_encoder = OneHotEncoder(sparse=False)
label_df = pd.DataFrame(label, columns = ['diagnosis'])
one_hot_encoder.fit(label_df)

label_df_encoded = one_hot_encoder.transform(label_df)
label_df_encoded = pd.DataFrame(data=label_df_encoded, columns=one_hot_encoder.categories_)
print(label_df_encoded.columns)

MultiIndex([('atypical melanocytic proliferation',),
            (               'cafe-au-lait macule',),
            (                       'lentigo NOS',),
            (               'lichenoid keratosis',),
            (                          'melanoma',),
            (                             'nevus',),
            (              'seborrheic keratosis',),
            (                     'solar lentigo',),
            (                           'unknown',)],
           )


In [7]:
df_train = pd.DataFrame(train_data)
df_train.columns = ['images']
df_train = pd.concat([df_train, label_df_encoded], axis = 1)
print(df_train.columns)
df_test = pd.DataFrame(test_data)
df_test.columns = ['images']

Index([                               'images',
       ('atypical melanocytic proliferation',),
                      ('cafe-au-lait macule',),
                              ('lentigo NOS',),
                      ('lichenoid keratosis',),
                                 ('melanoma',),
                                    ('nevus',),
                     ('seborrheic keratosis',),
                            ('solar lentigo',),
                                  ('unknown',)],
      dtype='object')


In [8]:
X_train, X_test, y_train, y_test = train_test_split(df_train.iloc[:, 0], df_train.iloc[:,1::], test_size = 0.3, random_state = 42)

train = pd.DataFrame(X_train)
train.columns = ['images']
train = pd.concat([train, y_train], axis = 1)

test = pd.DataFrame(X_test)
test.columns = ['images']
test = pd.concat([test, y_test], axis = 1)

In [9]:
train.columns[1:]

Index([('atypical melanocytic proliferation',),
                      ('cafe-au-lait macule',),
                              ('lentigo NOS',),
                      ('lichenoid keratosis',),
                                 ('melanoma',),
                                    ('nevus',),
                     ('seborrheic keratosis',),
                            ('solar lentigo',),
                                  ('unknown',)],
      dtype='object')

In [10]:
train_datagen = ImageDataGenerator(rescale=1./255,rotation_range=50,
                                   zoom_range = 0.3,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range = 0.2,
                                   horizontal_flip=True,
                                   fill_mode = 'nearest')
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_dataframe(
    train,
    x_col=train.columns[0],
    y_col=train.columns[1:],
    target_size=(224, 224),
    batch_size=8,
    shuffle=True,
    class_mode='raw')

test_generator = test_datagen.flow_from_dataframe(
    test,
    x_col=test.columns[0],
    y_col=test.columns[1:],
    target_size=(224, 224),
    shuffle=False,
    batch_size=8,
    class_mode='raw')

train_generator_effB3 = train_datagen.flow_from_dataframe(
    train,
    x_col=train.columns[0],
    y_col=train.columns[1:],
    target_size=(300, 300),
    batch_size=8,
    shuffle=True,
    class_mode='raw')

test_generator_effB3 = test_datagen.flow_from_dataframe(
    test,
    x_col=test.columns[0],
    y_col=test.columns[1:],
    target_size=(300, 300),
    shuffle=False,
    batch_size=8,
    class_mode='raw')

Found 23188 validated image filenames.
Found 9938 validated image filenames.
Found 23188 validated image filenames.
Found 9938 validated image filenames.


In [11]:
#Models

vgg16 = keras.models.load_model("Models/vgg16_1.h5")
vgg19 = keras.models.load_model("Models/vgg19.h5")
dense_169 = keras.models.load_model("Models/dense-169.h5")
dense_201 = keras.models.load_model("Models/dense-201.h5")
resnet_50 = keras.models.load_model("Models/resnet-50.h5")
resnet_152 = keras.models.load_model("Models/resnet-152.h5")
effB3 = keras.models.load_model("Models/efficientB3_1.h5")

2022-01-11 18:44:22.434025: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-01-11 18:44:22.434267: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-01-11 18:44:22.434418: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-11 18:44:22.434901: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce GTX 1080 Ti computeCapability: 6.1
coreClock: 1.582GHz coreCount: 28 deviceMemorySize: 10.91GiB deviceMemoryBandwidth: 451.17GiB/s
2022-01-11 18:44

In [12]:
def predict(models):
    pred_labels = []
    for model in models:
        test_datagen = ImageDataGenerator(rescale=1./255)
        data = None
        if model == effB3:
            data = test_datagen.flow_from_dataframe(dataframe = test,
              x_col="images", class_mode=None, 
              batch_size=8, target_size=(300,300), shuffle=False)
        else:
            data = test_datagen.flow_from_dataframe(dataframe = test,
              x_col="images", class_mode=None, 
              batch_size=8, target_size=(224,224), shuffle=False)
            
        prediction = model.predict_generator(data, max_queue_size=64, workers = 32, verbose=1)
        labels = []
        for _ in prediction:
            predict = np.array(_)
            label = np.zeros(_.shape)
            label.itemset(np.argmax(predict), 1)
            labels.append(label)
        pred_labels.append(labels)
    
    return np.array(pred_labels)
        

In [13]:
models = [vgg16, vgg19, dense_169, dense_201, resnet_50, resnet_152, effB3]
pred_labels = predict(models)
print(pred_labels.shape)

Found 9938 validated image filenames.


/home/deeplab/anaconda3/envs/tensorflow/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1905: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '
2022-01-11 18:44:36.190761: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-01-11 18:44:36.210055: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 4200000000 Hz
2022-01-11 18:44:36.304550: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.11
2022-01-11 18:44:36.494123: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublasLt.so.11
2022-01-11 18:44:38.442046: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully open

1243/1243 [==============================] - 118s 91ms/step
Found 9938 validated image filenames.
1243/1243 [==============================] - 115s 91ms/step
Found 9938 validated image filenames.
1243/1243 [==============================] - 119s 92ms/step
Found 9938 validated image filenames.
1243/1243 [==============================] - 119s 93ms/step
Found 9938 validated image filenames.
1243/1243 [==============================] - 116s 91ms/step
Found 9938 validated image filenames.
1243/1243 [==============================] - 116s 90ms/step
Found 9938 validated image filenames.
1243/1243 [==============================] - 116s 90ms/step
(7, 9938, 9)


In [14]:
ensemble_pred_labels = np.mean(pred_labels, axis=0)
ensemble_pred_labels.shape

(9938, 9)

In [15]:
ensemble_cls_pred = np.argmax(ensemble_pred_labels, axis=1)
ensemble_cls_pred.shape

(9938,)

In [16]:
ensemble_correct = []
for i in range(test.iloc[:, 1::].shape[0]):
    actual = np.argmax(test.iloc[i, 1::])
    ensemble_correct.append(actual == ensemble_cls_pred[i])

ensemble_incorrect = np.logical_not(ensemble_correct)

In [17]:
#Evaluate

scores_vgg16 = vgg16.evaluate(test_generator)
print("%s%s: %.2f%%" % ("evaluate ",vgg16.metrics_names[1], scores_vgg16[1]*100))

scores_vgg16_gen = vgg16.evaluate_generator(test_generator)
print("%s%s: %.2f%%" % ("evaluate_generator ",vgg16.metrics_names[1], scores_vgg16_gen[1]*100))

1243/1243 [==============================] - 355s 286ms/step - loss: 0.2352 - accuracy: 0.9467
evaluate accuracy: 94.67%


/home/deeplab/anaconda3/envs/tensorflow/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1877: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '


evaluate_generator accuracy: 94.67%


In [18]:
#Evaluate

scores_vgg19 = vgg19.evaluate(test_generator)
print("%s%s: %.2f%%" % ("evaluate ",vgg19.metrics_names[1], scores_vgg19[1]*100))

scores_vgg19_gen = vgg19.evaluate_generator(test_generator)
print("%s%s: %.2f%%" % ("evaluate_generator ",vgg19.metrics_names[1], scores_vgg19_gen[1]*100))

1243/1243 [==============================] - 333s 268ms/step - loss: 0.3043 - accuracy: 0.9417
evaluate accuracy: 94.17%
evaluate_generator accuracy: 94.17%


In [19]:
#Evaluate

scores_dense169 = dense_169.evaluate(test_generator)
print("%s%s: %.2f%%" % ("evaluate ",dense_169.metrics_names[1], scores_dense169[1]*100))

scores_dense169_gen = dense_169.evaluate_generator(test_generator)
print("%s%s: %.2f%%" % ("evaluate_generator ",dense_169.metrics_names[1], scores_dense169_gen[1]*100))

1243/1243 [==============================] - 350s 281ms/step - loss: 0.2730 - accuracy: 0.9288
evaluate accuracy: 92.88%
evaluate_generator accuracy: 92.88%


In [20]:
#Evaluate

scores_dense201 = dense_201.evaluate(test_generator)
print("%s%s: %.2f%%" % ("evaluate ",dense_201.metrics_names[1], scores_dense201[1]*100))

scores_dense201_gen = dense_201.evaluate_generator(test_generator)
print("%s%s: %.2f%%" % ("evaluate_generator ",dense_201.metrics_names[1], scores_dense201_gen[1]*100))

1243/1243 [==============================] - 352s 282ms/step - loss: 0.3105 - accuracy: 0.9175
evaluate accuracy: 91.75%
evaluate_generator accuracy: 91.75%


In [21]:
#Evaluate

scores_resnet50 = resnet_50.evaluate(test_generator)
print("%s%s: %.2f%%" % ("evaluate ",resnet_50.metrics_names[1], scores_resnet50[1]*100))

scores_resnet50_gen = resnet_50.evaluate_generator(test_generator)
print("%s%s: %.2f%%" % ("evaluate_generator ",resnet_50.metrics_names[1], scores_resnet50_gen[1]*100))

1243/1243 [==============================] - 352s 283ms/step - loss: 0.2334 - accuracy: 0.9492
evaluate accuracy: 94.92%
evaluate_generator accuracy: 94.92%


In [22]:
#Evaluate

scores_resnet152 = resnet_152.evaluate(test_generator)
print("%s%s: %.2f%%" % ("evaluate ",resnet_152.metrics_names[1], scores_resnet152[1]*100))

scores_resnet152_gen = resnet_152.evaluate_generator(test_generator)
print("%s%s: %.2f%%" % ("evaluate_generator ",resnet_152.metrics_names[1], scores_resnet152_gen[1]*100))

1243/1243 [==============================] - 357s 286ms/step - loss: 0.2857 - accuracy: 0.9463
evaluate accuracy: 94.63%
evaluate_generator accuracy: 94.63%


In [23]:
#Evaluate

scores_effB3 = effB3.evaluate(test_generator_effB3)
print("%s%s: %.2f%%" % ("evaluate ",effB3.metrics_names[1], scores_effB3[1]*100))

scores_effB3_gen = effB3.evaluate_generator(test_generator_effB3)
print("%s%s: %.2f%%" % ("evaluate_generator ",effB3.metrics_names[1], scores_effB3_gen[1]*100))

1243/1243 [==============================] - 364s 292ms/step - loss: 0.2395 - accuracy: 0.9415
evaluate accuracy: 94.15%
evaluate_generator accuracy: 94.15%


In [31]:
test_accuracies = [
    scores_vgg16[1], scores_vgg19[1], 
    scores_dense169[1], scores_dense201[1], 
    scores_resnet50[1], scores_resnet152[1], 
    scores_effB3[1]]

In [46]:
#Average Ensemble Accuracy
np.mean(test_accuracies)

0.9388063209397453

In [33]:
best_net = np.argmax(test_accuracies)
best_net

4

In [34]:
test_accuracies[best_net]

0.9491849541664124

In [35]:
best_net_pred_labels = pred_labels[best_net, :, :]

In [36]:
best_net_cls_pred = np.argmax(best_net_pred_labels, axis=1)

In [37]:
best_net_cls_pred

array([5, 8, 8, ..., 8, 8, 8])

In [38]:
best_net_correct = []
for i in range(test.iloc[:, 1::].shape[0]):
    actual = np.argmax(test.iloc[i, 1::])
    best_net_correct.append(actual == best_net_cls_pred[i])

best_net_incorrect = np.logical_not(best_net_correct)

In [39]:
np.sum(ensemble_correct)

9425

In [40]:
np.sum(best_net_correct)

9433

In [41]:
# Topluluğun doğru bulup, en iyi modelin yanlış buldukları
ensemble_better = np.logical_and(best_net_incorrect,
                                 ensemble_correct)
ensemble_better

array([False, False, False, ..., False, False, False])

In [42]:
ensemble_better.sum()

25

In [43]:
# En iyi modelin doğru bulup, topluluğun yanlış buldukları
best_net_better = np.logical_and(best_net_correct,
                                 ensemble_incorrect)

In [44]:
best_net_better.sum()

33